In [12]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output, State
import base64

import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps


from animal_shelter import AnimalShelter





###########################
# Data Manipulation / Model
###########################

username = "aacuser"
password = "pass"
shelter = AnimalShelter(username, password)


# class read method must support return of cursor object 
df = pd.DataFrame.from_records(shelter.readAll({}))



#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

image_filename = 'Grazioso Salvare Logo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))),
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard'))),
    html.Hr(),
    html.Div(
        
#FIXME Add in code for the interactive filtering options. For example, Radio buttons, drop down, checkboxes, etc.
    className='row',
        style={'display':'flex'},
        children=[
            html.Button(id='button-one', n_clicks=0, children='Water Rescue'),
            html.Button(id='button-two', n_clicks=0, children='Mountain/Wilderness Rescue'),
            html.Button(id='button-three', n_clicks=0, children='Disaster/Individual Tracking'),
            html.Button(id='button-four', n_clicks=0, children='Reset')
        ]


    ),
    html.Hr(),
    dt.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        style_cell={'textAlign': 'left'},
        editable=False,
        sort_action="native",
        sort_mode="multi",
        column_selectable=False,
        row_selectable=False,
        row_deletable=False,
        selected_columns=[],
        selected_rows=[],
        page_action="native",
        page_current= 0,
        page_size= 8,
    ), 
    
    html.H4('Created By: Eduardo Orozco'),
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',
            ),

        html.Div(
            dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            dl.Marker(position=[30.75,-97.48], children=[
                dl.Tooltip(df.iloc[0,4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(df.iloc[1,9])
                ])
            ])
        ]),
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################
@app.callback(Output('datatable-id',"data"), 
              [Input('button-one', 'n_clicks'),Input('button-two', 'n_clicks'),
               Input('button-three', 'n_clicks'),Input('button-four', 'n_clicks')])
def on_click(bt1,bt2,bt3,bt4):
    # start case
    if (int(bt1) >= 1):
        df = pd.DataFrame.from_records(shelter.readAll({
            "animal_type":"Dog", "breed":{"$in":["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"]},
            "sex_upon_outcome":"Intact Female","age_upon_outcome_in_weeks":{"$gte":26},
            "age_upon_outcome_in_weeks":{"$lte":156}}))
        
    elif (int(bt2) >= 1):
        df = pd.DataFrame.from_records(shelter.readAll({
            "animal_type":"Dog", "breed":{"$in":["German Sheperd", "Alaskan Malamute", "Old English Sheepdog",
                                                "Siberian Husky","Rottweiler"]},
            "sex_upon_outcome":"Intact Male","age_upon_outcome_in_weeks":{"$gte":26},
            "age_upon_outcome_in_weeks":{"$lte":156}}))
        
    elif (int(bt3) >=1):
        df = pd.DataFrame.from_records(shelter.readAll({
            "animal_type":"Dog", "breed":{"$in":["Doberman Pinscher", "German Sheperd", "Golden Retriever",
                                                "Bloodhound", ""]},
            "sex_upon_outcome":"Intact Male","age_upon_outcome_in_weeks":{"$gte":20},
            "age_upon_outcome_in_weeks":{"$lte":300}}))
    
        
    return df.to_dict('records')


@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
) 
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_graphs(viewData):
    df = pd.DataFrame.from_dict(viewData)
    return [
        dcc.Graph(
        figure = px.pie(df, names='breed', title='Percentage of breeds available')
        )
    ]

@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data"),
    Input('datatable-id',"derived_viewport_selected_rows")])
def update_map(viewData):
    dff = pd.Dataframe.from_dict(viewdata)
    return [
            dl.Map(style={'width': '1000px', 'height': '500px'}, center=
            [dff.loc[0,'location_lat'],loc[0,'location_long']], zoom=15, children=[
            dl.TileLayer(id="base-layer-id"),
            dl.Marker(position=[dff.loc[0,'location_lat'],dff.loc[0,'location_long']], children=[
                dl.Tooltip(dff['breed']),
                dl.Popup([
                    html.H1(dff['breed']),
                    html.P(dff.iloc[0,'name'])
                ])
            ])
        ])]
       
    



app